# Tugas Besar II
## Feed Forward Neural Network
### IF4071 Pembelajaran Mesin

### 1.a Create Classifier
 
Membuat kode program Backpropagation (lihat contoh algoritma di buku Tom Mitchell hal. 98), dengan spesifikasi berikut:
* Jumlah hidden layer maksimal 10
* Jumlah node dalam setiap hidden layer dapat bervariasi
* Fully-connected layer
* Fungsi aktivasi berupa sigmoid untuk semua hidden layer maupun output layer
* Node output berjumlah 1
* Program memberikan pilihan untuk menggunakan momentum atau tidak
* Program mengimplementasikan mini-batch stochastic gradient descent
* Implementasi incremental dengan setting batch-size=1 dan implementasi batch dengan setting batch-size=jumlah data.

In [100]:
"""
network.py
~~~~~~~~~~
A module to implement the stochastic gradient descent learning
algorithm for a feedforward neural network.  Gradients are calculated
using backpropagation.  Note that I have focused on making the code
simple, easily readable, and easily modifiable.  It is not optimized,
and omits many desirable features.
"""

#### Libraries
# Standard library
import random

# Third-party libraries
import numpy as np

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
            # feedforward
        activation = a
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b.transpose()[0]
            activation = sigmoid(z)
        return activation

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
#             random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
#             if test_data:
#                 print("Epoch " +  str(j) + ": ")
#                 print(self.evaluate(test_data))
#                 print("/" +  str(n_test))
#             else:
#                 print("Epoch " + str(j) + " complete")
        print(self.evaluate(test_data))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
#         print("FORWARD")
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b.transpose()[0]
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        
#         print("BACKWARD")
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = multiply(delta, activations[-2])
#         print("delta")
#         print(delta)
#         print("activation")
#         print(activations[-2])
#         print("nabla_w[-1]")
#         print(nabla_w[-1])
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = multiply(delta, activations[-l-1].transpose())
#             print("delta")
#             print(delta)
#             print("activation")
#             print(activations[-l-1])
#             print("nabla_w[-l]")
#             print(nabla_w[-l])
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(self.feedforward(x), y)
                        for (x, y) in test_data]
        print(test_results)
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return np.squeeze(output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

def multiply(A, B):
    result = []
    for i in range(len(A)) :
        row = []
        for j in range(len(B)):
            row.append(A[i]*B[j])
        result.append(row)
        
    return row
    

In [21]:
a = np.array([i for i in range(20)])
# a = np.zeros((10, 2))
# a = a.reshape(2,-1)
print(a)
print(a.shape)
print(np.multiply(a,2))
b = a.T
print(b.shape)

print(np.dot(a,b))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
(20,)
[ 0  2  4  6  8 10 12 14 16 18 20 22 24 26 28 30 32 34 36 38]
(20,)
2470


In [107]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import shutil

import numpy as np

from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split
# from tensorflow.contrib import skflow

iris = datasets.load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris.data[0:100], iris.target[0:100],
    test_size=0.2, random_state=42)
print(y_train)
train_data = [(x, y) for x, y in zip(X_train, y_train)]
test_data = [(x, y) for x, y in zip(X_test, y_test)]

neural_network = Network([4,5,10,11,1])
# neural_network = Network([2,2,1])
# neural_network.biases[0][0][0] = 0.1
# neural_network.biases[0][1][0] = 0.1
# neural_network.biases[1][0][0] = 0.2
# print(neural_network.biases)
# neural_network.weights[1][0][0] = 0.2
# neural_network.weights[1][0][1] = 0.3
# neural_network.weights[0][0][0] = -0.2
# neural_network.weights[0][0][1] = 0.1
# neural_network.weights[0][1][0] = -0.1
# neural_network.weights[0][1][1] = 0.3
# print("WEIGHTS")
# print(neural_network.weights)
# print(train_data)
# neural_network.SGD([(np.array([0.1, 0.9]), 0.9)], 1, 1, 0.001)
neural_network.SGD(train_data, 10000, 5, 0.1, test_data)

# check model load with test data
# score = metrics.accuracy_score(y_test, new_classifier.predict(X_test))
# print('Accuracy: {0:f}'.format(score))

[1 1 0 0 1 0 0 1 0 1 0 0 1 0 1 0 1 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0
 1 0 1 1 1 1 0 1 1 0 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 0 0 0 1 0 0 0 1 1 1 1 1
 0 1 1 0 1 1]
[(array([0.99679859]), 1), (array([0.99529998]), 1), (array([0.99676112]), 1), (array([0.00224749]), 0), (array([0.14219284]), 0), (array([0.00316446]), 0), (array([0.0023155]), 0), (array([0.99321674]), 1), (array([0.00343931]), 0), (array([0.00285197]), 0), (array([0.0056686]), 0), (array([0.00387219]), 0), (array([0.99673424]), 1), (array([0.00562178]), 0), (array([0.99669301]), 1), (array([0.00383435]), 0), (array([0.99657072]), 1), (array([0.99681464]), 1), (array([0.00198671]), 0), (array([0.00263613]), 0)]
0
